In this Python notebook, we will be employing transfer learning techniques to train existing models (AlexNet, VGG, and DenseNet). The process will be divided into two main parts:

Feature Extraction: In the first part, we will utilize the models as fixed feature extractors. This involves modifying only the last fully connected (FC) layer of each model to suit our specific task and training just this layer while keeping the rest of the model unchanged.

Fine-Tuning: In the second part, we will implement fine-tuning, which involves training all the layers of the model on our dataset, starting from the pretrained weights. We will specifically apply this approach to AlexNet.

In [133]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy

# pytorch imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchvision import models, transforms
import loralib as lora

In [134]:
# create train dataloader and valid dataloader
def set_transform():
  train_transform = transforms.Compose([
      transforms.RandomResizedCrop(224),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

  valid_transform = transforms.Compose([
      transforms.RandomResizedCrop(224),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

  return train_transform, valid_transform

def create_dataloader(root_dir, batch_size):
  train_transform, valid_transform = set_transform()
  # Create datasets
  train_dataset = ImageFolder(root=root_dir + '/train', transform=train_transform)
  valid_dataset = ImageFolder(root=root_dir + '/val', transform=valid_transform)
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=2, shuffle=True)
  valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, num_workers=2, shuffle=False)
  dataloaders = {'train' : train_loader, 'val' : valid_loader}
  return dataloaders

In [135]:
def set_parameter_requires_grad(model, feature_extracting=False):
    """
    Sets the .requires_grad attribute of the parameters in the model. This is used to freeze or unfreeze
    the model's parameters during training.

    Parameters:
    - model (torch.nn.Module): The model whose parameters will be modified.
    - feature_extracting (bool): If True, the model is being used for feature extraction, and all
      parameters are frozen to prevent gradients from being computed. If False, the model will be fine-tuned,
      and gradients will be computed for all parameters.

    If feature_extracting is True, this function goes through all parameters in the model and sets
    .requires_grad to False, effectively freezing the model during training. This is useful when you're only
    interested in training a few top layers of a model while keeping the rest unchanged.

    Conversely, if feature_extracting is False, this indicates that the model will undergo fine-tuning,
    where all model parameters are subject to training. Therefore, it sets .requires_grad to True for all
    parameters, allowing gradients to be computed and weights to be updated during training.
    """
    if feature_extracting:
        # Freeze all parameters for feature extraction
        for param in model.parameters():
            param.requires_grad = False
    else:
        # Unfreeze all parameters for fine-tuning
        for param in model.parameters():
            param.requires_grad = True


In [136]:
import torchvision.models as models
import torch.nn as nn

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True, set_lora=False):
    """
    Initializes a model for transfer learning based on the specified architecture (model_name).
    This function can adjust models for a new task by changing their output layers to match
    the number of classes for the new task (num_classes). It supports operating in either
    feature extraction or fine-tuning mode.

    Parameters:
    - model_name (str): The name of the model architecture to be initialized.
    - num_classes (int): The number of classes for the new task.
    - feature_extract (bool): If True, the model is set to feature extraction mode where only the final layer's parameters are updated.
    - use_pretrained (bool): If True, initializes the model with weights pre-trained on ImageNet.
    - set_lora (bool): Placeholder for future use, not implemented in this function.

    Returns:
    - model_ft (torch.nn.Module): The initialized model ready for training on the new task.
    - input_size (int): The required input image size for the model, e.g., 224 for AlexNet.
    """
    
    # Initialize variables for the model and input size.
    model_ft = None
    input_size = 0

    # Define weights setting based on whether pretrained weights are desired.
    weights = 'DEFAULT' if use_pretrained else None

    if model_name == "resnet":
        # Initialize ResNet
        model_ft = models.resnet18(weights=weights)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        # Initialize AlexNet
        model_ft = models.alexnet(weights=weights)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features       
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_classes)
        input_size = 224
        
    elif model_name == "vgg":
        # Initialize VGG
        model_ft = models.vgg16(weights=weights)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        # Initialize SqueezeNet
        model_ft = models.squeezenet1_0(weights=weights)
        set_parameter_requires_grad(model_ft, feature_extract)
        # SqueezeNet uses a Conv2d layer as its classifier.
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        # Initialize DenseNet
        model_ft = models.densenet121(weights=weights)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        # Note: DenseNet's input size is often larger in practice, but 224 works for compatibility.
        input_size = 224

    else:
        raise NotImplementedError("The specified model name is not supported.")

    return model_ft, input_size

In [137]:
def params_to_update(model, feature_extract):
    """
    Identifies and returns the parameters to be optimized/updated in this run. If we are
    fine-tuning, we will update all parameters. However, if we are using the feature extraction
    method, we will only update the parameters that were recently initialized, i.e., the parameters
    for which requires_grad is True.

    Parameters:
    - model (torch.nn.Module): The model for which parameters are to be identified.
    - feature_extract (bool): Flag indicating whether we are feature extracting. If True, only parameters
      that require gradients are updated; otherwise, all parameters are updated.

    Returns:
    - params_to_update (list of torch.nn.parameter.Parameter): A list of parameters that will be updated.
    """
    if feature_extract:
        params_to_update = []
        print("Params to learn:")
        for name, param in model.named_parameters():
            if param.requires_grad:
                params_to_update.append(param)
                print("\t", name)
    else:
        # In the case of fine-tuning, we consider all parameters for updates,
        # but still filter them based on requires_grad to be consistent with the condition.
        # This allows the function to be flexible for models where some parameters might explicitly be frozen.
        params_to_update = [param for param in model.parameters() if param.requires_grad]
        print("Params to learn:")
        for name, param in model.named_parameters():
            if param.requires_grad:
                print("\t", name)

    return params_to_update


In [81]:
import time
import copy
import torch
import os

def train_model(model, model_name, dataloaders, criterion, optimizer, num_epochs=25, set_lora=False):
    """
    Train and validate a PyTorch model.

    Parameters:
    - model: The model to be trained and validated.
    - model_name: Name of the model, used for saving checkpoints.
    - dataloaders: A dictionary containing 'train' and 'val' DataLoaders for loading the data.
    - criterion: The loss function.
    - optimizer: The optimization algorithm.
    - num_epochs (int): The number of epochs to train the model.
    - set_lora (bool): Flag for potential future use, not used in the current implementation.

    Returns:
    - model: The trained model.
    - val_acc_history: A list with validation accuracy for each epoch.
    """

    since = time.time()

    val_acc_history = []

    # Store the original model parameters to restore later if needed.
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch goes through both training and validation phases
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data in batches.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                # Track history if in train phase
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    # Backward pass and optimize if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # Calculate epoch loss and accuracy
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Save the model if it has the best validation accuracy so far
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                results = [model_name, 'validation data', epoch, epoch_acc.item()]
                val_acc_history.append(results)

        print()

    # Print out the training time and best validation accuracy
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights and save the model checkpoint
    model.load_state_dict(best_model_wts)
    if not os.path.isdir('checkpoints'):
        os.mkdir('checkpoints')
    torch.save({'net': model.state_dict()}, f'./checkpoints/{model_name}.pth')

    return model, val_acc_history


In [82]:
import numpy as np
import torch

def calculate_accuracy(model, dataloader, device):
    """
    Calculate the accuracy of a PyTorch model on a given dataset.

    Parameters:
    - model (torch.nn.Module): The model to evaluate.
    - dataloader (torch.utils.data.DataLoader): DataLoader for the dataset to evaluate the model on.
    - device (torch.device): The device on which to perform the calculations (e.g., 'cuda' or 'cpu').

    Returns:
    - model_accuracy (float): The accuracy of the model on the dataset as a percentage.
    - confusion_matrix (np.array): A confusion matrix of size [num_classes, num_classes], where
      num_classes is the number of unique labels in the dataset. The matrix is used to evaluate
      the model's performance across different classes.
    """

    # Switch the model to evaluation mode to deactivate Dropout and use learned statistics in BatchNorm.
    model.eval()

    total_correct = 0
    total_images = 0
    # Assuming a maximum of 38 classes - adjust the size according to your specific task.
    confusion_matrix = np.zeros([38, 38], int)

    with torch.no_grad():  # Disable gradient computation for efficiency
        for data in dataloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass to get the model's predictions
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            # Update total images and correctly predicted counts
            total_images += labels.size(0)
            total_correct += (predicted == labels).sum().item()

            # Update confusion matrix
            for i, l in enumerate(labels):
                confusion_matrix[l.item(), predicted[i].item()] += 1

    # Calculate accuracy as the percentage of correct predictions
    model_accuracy = total_correct / total_images * 100

    return model_accuracy, confusion_matrix

In [50]:
'''
Main setup for running VGG as a feature extractor
'''
root_dir = r'/Users/noambuzaglo/Desktop/lilush/PlantVillage'
batch_size = 4
dataloaders = create_dataloader(root_dir, batch_size)
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet]
model_name = "vgg"
# Number of classes in the dataset
num_classes = 38
# Batch size for training (change depending on how much memory you have)
batch_size = 8
# Number of epochs to train for
num_epochs = 15
# Flag for feature extracting. When False, we fine-tune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# Print the model we just instantiated
print(model_ft)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
params_to_update = params_to_update(model_ft)
if torch.backends.mps.is_built():
    print("using MPS")
    device = torch.device("mps")
model_ft = model_ft.to(device)


optimizer_ft = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)
# Setup the loss fn
criterion = nn.CrossEntropyLoss()
# Train and evaluate
model_ft, hist = train_model(model_ft, model_name, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [51]:
"""
save validation result to CSV
""" 
import csv
filename = f"results_{model_name}.csv"
with open (filename, 'w', newline='') as f:
    writer = csv.writer(f)
    for row in hist:
        writer.writerow(row)

In [53]:
"""
Evaluating the trained model on the test dataset reveals its accuracy and performance, offering insights into how well it generalizes to new, unseen data.
""" 
test_transform = transforms.Compose([
      transforms.RandomResizedCrop(224),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])
test_dataset = ImageFolder(root=root_dir + '/test', transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=2, shuffle=True)

test_accuracy, _ = calculate_accuracy(model_ft, test_loader, device)
print(f"test accuracy:{test_accuracy}")


test accuracy:84.75046210720888


In [56]:
'''
Main setup for running alexnet as a feature extractor
'''
root_dir = r'/Users/noambuzaglo/Desktop/lilush/PlantVillage'
batch_size = 4
dataloaders = create_dataloader(root_dir, batch_size)
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet]
model_name = "alexnet"
# Number of classes in the dataset
num_classes = 38
# Batch size for training (change depending on how much memory you have)
batch_size = 8
# Number of epochs to train for
num_epochs = 15
# Flag for feature extracting. When False, we fine-tune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# Print the model we just instantiated
print(model_ft)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
params_to_update = params_to_update(model_ft)
if torch.backends.mps.is_built():
    print("using MPS")
    device = torch.device("mps")
model_ft = model_ft.to(device)
optimizer_ft = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)
##%%
# Setup the loss fn
criterion = nn.CrossEntropyLoss()
# Train and evaluate
model_ft, hist = train_model(model_ft, model_name, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [57]:
"""
save validation result to CSV
""" 

filename = f"results_{model_name}.csv"
with open (filename, 'w', newline='') as f:
    writer = csv.writer(f)
    for row in hist:
        writer.writerow(row)
"""
Evaluating the trained model on the test dataset reveals its accuracy and performance, offering insights into how well it generalizes to new, unseen data.
""" 
test_accuracy, _ = calculate_accuracy(model_ft, test_loader, device)
print(f"test accuracy:{test_accuracy}")

test accuracy:84.84288354898337


In [91]:
'''
Main setup for running densenet as a feature extractor
'''
root_dir = r'/Users/noambuzaglo/Desktop/lilush/PlantVillage'
batch_size = 4
dataloaders = create_dataloader(root_dir, batch_size)
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet]
model_name = "densenet"
# Number of classes in the dataset
num_classes = 38
# Batch size for training (change depending on how much memory you have)
batch_size = 8 
# Number of epochs to train for
num_epochs = 15
# Flag for feature extracting. When False, we fine-tune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# Print the model we just instantiated
print(model_ft)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
params_to_update = params_to_update(model_ft)
if torch.backends.mps.is_built():
    print("using MPS")
    device = torch.device("mps")
model_ft = model_ft.to(device)
optimizer_ft = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)
##%%
# Setup the loss fn
criterion = nn.CrossEntropyLoss()
# Train and evaluate
model_ft, hist = train_model(model_ft, model_name, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [92]:
"""
save validation result to CSV
""" 
filename = f"results_{model_name}.csv"
with open (filename, 'w', newline='') as f:
    writer = csv.writer(f)
    for row in hist:
        writer.writerow(row)

"""
Evaluating the trained model on the test dataset reveals its accuracy and performance, offering insights into how well it generalizes to new, unseen data.
""" 
test_accuracy, _ = calculate_accuracy(model_ft, test_loader, device)
print(f"test accuracy:{test_accuracy}")

test accuracy:88.56284658040666


In [85]:
'''
Main setup for running alexnet as a fine-tuning
'''
root_dir = r'/Users/noambuzaglo/Desktop/lilush/PlantVillage'
batch_size = 128
dataloaders = create_dataloader(root_dir, batch_size)
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet]
model_name = "alexnet"
# Number of classes in the dataset
num_classes = 38
# Batch size for training (change depending on how much memory you have)
batch_size = 32
# Number of epochs to train for
num_epochs = 15
# Flag for feature extracting. When False, we fine-tune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
# Print the model we just instantiated
print(model_ft)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.backends.mps.is_built():
    print("using MPS")
    device = torch.device("mps")
model_ft = model_ft.to(device)
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
##%%
# Setup the loss fn
criterion = nn.CrossEntropyLoss()
# Train and evaluate
model_name = 'alexnet_fine_tune'
model_ft, hist = train_model(model_ft, model_name, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [86]:
filename = f"results_{model_name}.csv"
with open (filename, 'w', newline='') as f:
    writer = csv.writer(f)
    for row in hist:
        writer.writerow(row)

test_accuracy, _ = calculate_accuracy(model_ft, test_loader, device)
print(f"test accuracy:{test_accuracy}")

test accuracy:96.85767097966729


In [ ]:
'''
Main setup for running alexnet as a fine-tuning
'''
root_dir = r'/Users/noambuzaglo/Desktop/lilush/PlantVillage'
batch_size = 128
dataloaders = create_dataloader(root_dir, batch_size)
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet]
model_name = "alexnet"
# Number of classes in the dataset
num_classes = 38
# Batch size for training (change depending on how much memory you have)
batch_size = 32
# Number of epochs to train for
num_epochs = 15
# Flag for feature extracting. When False, we fine-tune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=False)
# Print the model we just instantiated
print(model_ft)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.backends.mps.is_built():
    print("using MPS")
    device = torch.device("mps")
model_ft = model_ft.to(device)
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
##%%
# Setup the loss fn
criterion = nn.CrossEntropyLoss()
# Train and evaluate
model_name = 'alexnet_fine_tune_with_weights'
model_ft, hist = train_model(model_ft, model_name, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

from here start lora part

In [138]:
model_ft = models.alexnet(weights='DEFAULT')
set_parameter_requires_grad(model_ft, True)
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = lora.Linear(num_ftrs, 38, 16)
# Your custom classifier
# Assuming num_classes is the number of output classes you have

for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
              print("\t",name)

lora.mark_only_lora_as_trainable(model_ft)
params_to_update = []  # override the initial list definition above
for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
optimizer = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)

	 classifier.6.bias
	 classifier.6.lora_A
	 classifier.6.lora_B


In [139]:
# main code:
root_dir = r'/Users/noambuzaglo/Desktop/lilush/PlantVillage'
batch_size = 128
dataloaders = create_dataloader(root_dir, batch_size)
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet]
model_name = "alexnet"
# Number of classes in the dataset
num_classes = 38
# Batch size for training (change depending on how much memory you have)
batch_size = 32
# Number of epochs to train for
num_epochs = 15
# Flag for feature extracting. When False, we fine-tune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True
# model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True, set_lora=True)
# Print the model we just instantiated
# print(model_ft)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# params_to_update = params_to_update(model_ft)
if torch.backends.mps.is_built():
    print("using MPS")
    device = torch.device("mps")
model_ft = model_ft.to(device)
optimizer_ft = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)
##%%
# Setup the loss fn
criterion = nn.CrossEntropyLoss()
# Train and evaluate
model_name = 'alexnet22'
model_ft, hist = train_model(model_ft, model_name, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs, set_lora=True)

using MPS
	 classifier.6.lora_A
	 classifier.6.lora_B
	 classifier.6.lora_A
	 classifier.6.lora_B
Epoch 0/14
----------
train Loss: 3.2468 Acc: 0.1732
val Loss: 2.5228 Acc: 0.3287

Epoch 1/14
----------
train Loss: 2.1291 Acc: 0.4092
val Loss: 1.6984 Acc: 0.5047

Epoch 2/14
----------
train Loss: 1.5662 Acc: 0.5399
val Loss: 1.2964 Acc: 0.6109

Epoch 3/14
----------
train Loss: 1.2600 Acc: 0.6211
val Loss: 1.0397 Acc: 0.6895

Epoch 4/14
----------
train Loss: 1.0818 Acc: 0.6735
val Loss: 0.9020 Acc: 0.7269

Epoch 5/14
----------
train Loss: 0.9911 Acc: 0.6983
val Loss: 0.8077 Acc: 0.7582

Epoch 6/14
----------
train Loss: 0.9191 Acc: 0.7188
val Loss: 0.7578 Acc: 0.7691

Epoch 7/14
----------
train Loss: 0.8636 Acc: 0.7354
val Loss: 0.7208 Acc: 0.7793

Epoch 8/14
----------
train Loss: 0.8286 Acc: 0.7447
val Loss: 0.6704 Acc: 0.7956

Epoch 9/14
----------
train Loss: 0.7908 Acc: 0.7579
val Loss: 0.6353 Acc: 0.8046

Epoch 10/14
----------
train Loss: 0.7569 Acc: 0.7654
val Loss: 0.6275 A

In [140]:
# main code:
root_dir = r'/Users/noambuzaglo/Desktop/lilush/PlantVillage'
batch_size = 4
dataloaders = create_dataloader(root_dir, batch_size)
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet]
model_name = "alexnet"
# Number of classes in the dataset
num_classes = 38
# Batch size for training (change depending on how much memory you have)
batch_size = 8
# Number of epochs to train for
num_epochs = 15
# Flag for feature extracting. When False, we fine-tune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True
# model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True, set_lora=True)
# Print the model we just instantiated
# print(model_ft)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# params_to_update = params_to_update(model_ft)
if torch.backends.mps.is_built():
    print("using MPS")
    device = torch.device("mps")
model_ft = model_ft.to(device)
optimizer_ft = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)
##%%
# Setup the loss fn
criterion = nn.CrossEntropyLoss()
# Train and evaluate
model_name = 'alexnet23'
model_ft, hist = train_model(model_ft, model_name, dataloaders, criterion, optimizer_ft, num_epochs=num_epochs, set_lora=True)

using MPS
	 classifier.6.lora_A
	 classifier.6.lora_B
	 classifier.6.lora_A
	 classifier.6.lora_B
Epoch 0/14
----------
train Loss: 0.6784 Acc: 0.7886
val Loss: 0.5005 Acc: 0.8486

Epoch 1/14
----------
train Loss: 0.6585 Acc: 0.7965
val Loss: 0.4884 Acc: 0.8494

Epoch 2/14
----------
train Loss: 0.6437 Acc: 0.7990
val Loss: 0.4909 Acc: 0.8473

Epoch 3/14
----------
train Loss: 0.6342 Acc: 0.8025
val Loss: 0.4617 Acc: 0.8566

Epoch 4/14
----------
train Loss: 0.6254 Acc: 0.8036
val Loss: 0.4709 Acc: 0.8526

Epoch 5/14
----------
train Loss: 0.6126 Acc: 0.8079
val Loss: 0.4643 Acc: 0.8611

Epoch 6/14
----------
train Loss: 0.6084 Acc: 0.8102
val Loss: 0.4432 Acc: 0.8593

Epoch 7/14
----------
train Loss: 0.6016 Acc: 0.8115
val Loss: 0.4430 Acc: 0.8604

Epoch 8/14
----------
train Loss: 0.5837 Acc: 0.8166
val Loss: 0.4314 Acc: 0.8658

Epoch 9/14
----------
train Loss: 0.5834 Acc: 0.8158
val Loss: 0.4240 Acc: 0.8690

Epoch 10/14
----------
train Loss: 0.5737 Acc: 0.8193
val Loss: 0.4136 A

In [141]:
test_transform = transforms.Compose([
      transforms.RandomResizedCrop(224),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])
test_dataset = ImageFolder(root=root_dir + '/test', transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=2, shuffle=True)

test_accuracy, _ = calculate_accuracy(model_ft, test_loader, device)
print(f"test accuracy:{test_accuracy}")


test accuracy:88.1931608133087
